In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
df = pd.read_excel('test-data.xlsx')

In [3]:
df.shape

(59998, 4)

59998 записей и четыре параметра

In [4]:
df.head()

,order_id,created,user_id,payment
0,3178039,2018-03-09 19:12:13,1066,1090
1,2940729,2018-02-02 13:56:01,4821,349
2,4260454,2018-08-14 13:32:25,4831,673
3,3468644,2018-04-24 14:24:58,9563,893
4,2842424,2018-01-15 15:44:33,12189,480


таблица заказов определяется номером заказа, временем, id пользователя и суммой

In [5]:
#отсортируем данные по времени
df_sorted = df.sort_values(by='created')

In [8]:
df_sorted.created.min(), df_sorted.created.max()

(Timestamp('2018-01-01 13:42:39'), Timestamp('2018-10-15 16:45:59'))

У нас данные с первого января 2018 по 15 октября 2018

In [9]:
#найдем номер недели для каждой транзакции
df_sorted['weekofyear'] = df_sorted.created.apply(lambda x: x.isocalendar()[1])

In [13]:
#найдем неделю первой покупки для каждого пользователя
first_user_week = pd.DataFrame(df_sorted.groupby(by='user_id')['weekofyear'].min()).reset_index()
first_user_week.columns =['user_id','first-user-week']

In [14]:
first_user_week.head()

,user_id,first-user-week
0,1066,10
1,4821,5
2,4831,33
3,9563,17
4,12189,3


In [15]:
#заджойним на исходную таблицу
new_df = df_sorted.merge(first_user_week, how='inner', on='user_id')

In [16]:
new_df.head()

,order_id,created,user_id,payment,weekofyear,first-user-week
0,2806102,2018-01-01 13:42:39,235067,340,1,1
1,4116816,2018-07-26 16:13:32,235067,282,30,1
2,2806119,2018-01-01 14:21:01,749173,680,1,1
3,3080702,2018-02-23 10:48:24,749173,680,8,1
4,3084503,2018-02-24 20:23:15,749173,680,8,1


In [17]:
#посчитаем третью неделю для каждого пользователя
new_df['3 weeks after'] = new_df['first-user-week'] + 2

In [29]:
new_df.head()

,order_id,created,user_id,payment,weekofyear,first-user-week,3 weeks after
0,2806102,2018-01-01 13:42:39,235067,340,1,1,3
1,4116816,2018-07-26 16:13:32,235067,282,30,1,3
2,2806119,2018-01-01 14:21:01,749173,680,1,1,3
3,3080702,2018-02-23 10:48:24,749173,680,8,1,3
4,3084503,2018-02-24 20:23:15,749173,680,8,1,3


In [27]:
#посчитаем сумму по третьим неделям
new_df[new_df['3 weeks after'] == new_df['weekofyear']].groupby(
    by='3 weeks after')['payment'].sum().sort_values(ascending=False).head()

3 weeks after
35    70802
33    64855
23    64383
37    58060
28    56648
Name: payment, dtype: int64

Мы получили, что на 35 й неделе сумма максимальная, значит клиенты, которые 
пришли на 33 й неделе принесли больше всего денег на третей неделе

In [28]:
new_df[new_df['3 weeks after'] == 35]

,order_id,created,user_id,payment,weekofyear,first-user-week,3 weeks after
52939,4245595,2018-08-13 09:09:59,1134156,224,33,33,35
52940,4245672,2018-08-13 09:18:58,1134172,614,33,33,35
52941,4245700,2018-08-13 09:22:33,1133425,322,33,33,35
52942,4245811,2018-08-13 09:35:01,1129601,744,33,33,35
52943,4245858,2018-08-13 09:39:30,1134198,408,33,33,35
52944,4430268,2018-09-03 09:23:26,1134198,370,36,33,35
52945,4245876,2018-08-13 09:41:39,1134201,411,33,33,35
52946,4245951,2018-08-13 09:49:09,1134214,300,33,33,35
52947,4246006,2018-08-13 09:54:57,1134218,351,33,33,35
52948,4246076,2018-08-13 10:02:10,1134236,407,33,33,35
